In [2]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [1]:
!pip install "apache_beam==2.60.0"

## **Partition**:

•	Partition is a Beam transform for PCollection objects that store the same data type. It splits a single PCollection into a fixed number of smaller collections.

•	Partition divides the elements of a PCollection according to a partitioning function that you provide.

•	The partitioning function contains the logic that determines how to split up the elements of the input PCollection into each resulting partition PCollection.

•	The number of partitions must be determined at graph construction time.

•	Partition accepts a function that receives the number of partitions, and returns the index of the desired partition for the element. The number of partitions passed must be a positive integer, and it must return an integer in the range 0 to num_partitions-1.



In [8]:
import apache_beam as beam

p = beam.Pipeline()
number = {11,12,13,44,55,61,77,88,99}

def partition_fn(element,num_partition):
  return 0 if element%2 ==0 else 1


number_pc = p| beam.Create(number)| beam.Partition(partition_fn,2)

number_pc[0]| 'Printing first partition' >> beam.Map(print)

# number_pc[1]| 'Printing second partition' >> beam.Map(print)

p.run()

12
44
88


## **Flatten**:

•	Flatten is a Beam transform for PCollection objects that store the same data type. Flatten merges multiple PCollection objects into a single logical PCollection.

•	Kind of Union operation


In [9]:
with beam.Pipeline() as pipeline:
  even_data = (pipeline
               |"Create even data" >> beam.Create({2,4,6,8,10}))
  odd_data = (pipeline
              |"Create odd data" >> beam.Create({1,3,5,7,9,11}))

  result = ((even_data, odd_data) | beam.Flatten()) | beam.Map(print)

1
3
5
7
9
11
2
4
6
8
10
